### 项目流程

1) 制作词向量，可以用gensim库，也可以用训练好的Word2Vec模型

2) 词和ID的映射

3) 构建LSTM网络架构

4) 训练模型

5) 结果分析

#### 导入数据

首先创建词向量：为了简单起见，我们使用Google在大规模数据集上训练好的Word2Vec 模型来创建。

在这个模型中，谷歌能创建 300 万个词向量，每个向量维度为 300。

但因为这个单词向量矩阵相当大（3.6G），我们用另外一个由 GloVe 进行训练到的小矩阵。

矩阵将包含 400000 个词向量，每个向量的维数为 50。

我们将导入两个不同的数据结构，一个是包含 400000 个单词的 Python 列表（wordList），一个是包含所有单词向量值得 400000*50 维的嵌入矩阵(wordVectors)。

In [1]:
import numpy as np
wordList = np.load('./data/training_data/wordsList.npy')
print('Loaded the word list!')
wordList = wordList.tolist()   # originally loaded as numpy array
wordList = [word.decode('UTF-8') for word in wordList]  # encode word as UFF-8
wordVectors = np.load('./data/training_data/wordVectors.npy')
print('Loaded the word vectors!')

Loaded the word list!
Loaded the word vectors!


In [2]:
print(len(wordList))   # 每个英文单词对应一个id的映射
print(wordVectors.shape)   # 词向量维度

400000
(400000, 50)


训练集我们使用的是 IMDB 数据集。这个数据集包含 25000 条电影数据，其中 12500 条正向数据，12500 条负向数据。正向数据包含在一个文本文件中，负向数据包含在另一个文本文件中。

在训练集上构造索引前，我们可先计算或可视化每个文件中单词的平均长度。这将帮助去决定如何设置最大序列长度的最佳值。

In [3]:
from os import listdir
from os.path import isfile, join
positiveFiles = ['./data/training_data/positiveReviews/' + f for f in listdir('./data/training_data/positiveReviews/') if isfile(join('./data/training_data/positiveReviews/', f))]
negativeFiles = ['./data/training_data/negativeReviews/' + f for f in listdir('./data/training_data/negativeReviews/') if isfile(join('./data/training_data/negativeReviews/', f))]
numWords = []
for pf in positiveFiles:
    with open(pf, "r", encoding='utf-8') as f:
        line=f.readline()
        counter = len(line.split())  # 空格分割
        numWords.append(counter)       
print('Positive files finished')

for nf in negativeFiles:
    with open(nf, "r", encoding='utf-8') as f:
        line=f.readline()
        counter = len(line.split())
        numWords.append(counter)  
print('Negative files finished')

numFiles = len(numWords)
print('The total number of files is', numFiles)
print('The total number of words in the files is', sum(numWords))
print('The average number of words in the files is', sum(numWords)/len(numWords))

Positive files finished
Negative files finished
The total number of files is 25000
The total number of words in the files is 5844680
The average number of words in the files is 233.7872


从句子的平均单词数，我们认为将句子最大长度设置为 250 是可行的。

In [4]:
maxSeqLength = 250

接下来我们将全部的 25000 条评论文件中的文本转换成索引矩阵（ 25000 * 250 ），即每个单词对应wordList中的一个索引

其中这些文本我们需要做数据清洗（预处理）操作，比如删除标点符号、去停用词等，只留下字母和数字字符

这是一个计算成本非常高的过程，可以直接使用处理好的索引矩阵文件。

In [5]:
# ids = np.zeros((numFiles, maxSeqLength), dtype='int32')
# fileCounter = 0
# for pf in positiveFiles:
#    with open(pf, "r") as f:
#        indexCounter = 0
#        line=f.readline()
#        cleanedLine = cleanSentences(line)
#        split = cleanedLine.split()
#        for word in split:
#            try:
#                ids[fileCounter][indexCounter] = wordsList.index(word)
#            except ValueError:
#                ids[fileCounter][indexCounter] = 399999 #Vector for unkown words
#            indexCounter = indexCounter + 1
#            if indexCounter >= maxSeqLength:
#                break
#        fileCounter = fileCounter + 1 

# for nf in negativeFiles:
#    with open(nf, "r") as f:
#        indexCounter = 0
#        line=f.readline()
#        cleanedLine = cleanSentences(line)
#        split = cleanedLine.split()
#        for word in split:
#            try:
#                ids[fileCounter][indexCounter] = wordsList.index(word)
#            except ValueError:
#                ids[fileCounter][indexCounter] = 399999 #Vector for unkown words
#            indexCounter = indexCounter + 1
#            if indexCounter >= maxSeqLength:
#                break
#        fileCounter = fileCounter + 1 
# #Pass into embedding function and see if it evaluates. 

# np.save('idsMatrix', ids)

In [6]:
ids = np.load('./data/training_data/idsMatrix.npy')
print(ids.shape)

(25000, 250)


#### 辅助函数

In [7]:
from random import randint

def getTrainBatch():
    labels = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        if (i % 2 == 0): 
            num = randint(1,11499)
            labels.append([1,0])
        else:
            num = randint(13499,24999)
            labels.append([0,1])
        arr[i] = ids[num-1:num]
    return arr, labels

def getTestBatch():
    labels = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        num = randint(11499,13499)
        if (num <= 12499):
            labels.append([1,0])
        else:
            labels.append([0,1])
        arr[i] = ids[num-1:num]
    return arr, labels

### LSTM模型构建

开始构建 TensorFlow 图模型：使用 TensorFlow 的嵌入函数，其有两个参数，一个是嵌入矩阵（在我们的情况下是词向量矩阵），另一个是每个词对应的索引。

In [13]:
batchSize = 24   # 批处理大小
lstmUnits = 64  # 隐藏层神经元个数
numClasses = 2   # 二分类情感分析
iterations = 50000   # 迭代次数
numDimensions = 300 #每个单词向量的维度

模型输入占位符是一个整数化的索引数组。

标签占位符代表一组值，每一个值都为 [1,0] 或者 [0,1]，这个取决于数据是正向的还是负向的。

In [25]:
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()
tf.reset_default_graph()
labels = tf.placeholder(tf.float32, [batchSize, numClasses])   # 标签占位符
input_data = tf.placeholder(tf.32, [batchSize, maxSeqLength])  # 输入数据占位符

设置输入数据占位符后，调用 tf.nn.embedding_lookup() 函数来得到词向量。

该函数将返回一个三维向量，第一个维度是批处理大小，第二个维度是句子长度，第三个维度是词向量长度。

In [26]:
data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)

现在已经得到了我们想要的数据形式，接下来将这种数据形式输入到 LSTM 网络中。

首先使用 tf.nn.rnn_cell.BasicLSTMCell 函数，其输入的参数是一个整数，表示需要几个 LSTM 单元。这是一个超参数，需要对这个数值进行调试从而找到最优的解。然后设置 dropout 参数避免过拟合。最后将 LSTM cell 和三维的数据输入到 tf.nn.dynamic_rnn ，这个函数的功能是展开整个网络，并且构建一整个 RNN 模型。

In [39]:
lstmCell = tf.nn.rnn_cell.LSTMCell(lstmUnits)   # 传入64个神经元
lstmCell = tf.compat.v1.nn.rnn_cell.DropoutWrapper(cell=lstmCell, output_keep_prob=0.75)   # 自定义的dropout
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)   # 模型展开

D:\Anaconda\envs\py37_tf\lib\site-packages\ipykernel_launcher.py:1: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  """Entry point for launching an IPython kernel.


dynamic RNN 函数的第一个输出可以被认为是最后的隐藏状态向量。这个向量将被重新确定维度，然后乘以最后的权重矩阵和一个偏置项来获得最终的输出值（全连接层）。

In [40]:
weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
#取最终的结果值
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

接下来定义正确的预测函数和正确率评估参数。正确的预测形式是查看最后输出的0-1向量是否和标记的0-1向量相同。

In [41]:
correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

之后使用标准的交叉熵损失函数来作为目标函数。优化器选择 Adam，并且采用默认的学习率。

In [43]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)

### 超参调整

训练损失值与选择的优化器（Adam，Adadelta，SGD，等等），学习率和网络架构都有很大的关系。特别是在RNN和LSTM中，单元数量和词向量的大小都是重要因素。

学习率：RNN最难的一点就是它的训练非常困难，因为时间步骤很长。那么，学习率就变得非常重要了。如果我们将学习率设置的很大，那么学习曲线就会波动性很大，如果我们将学习率设置的很小，那么训练过程就会非常缓慢。根据经验，将学习率默认设置为 0.001 是一个比较好的开始。如果训练的非常缓慢，那么你可以适当的增大这个值，如果训练过程非常的不稳定，那么你可以适当的减小这个值。

优化器：这个在研究中没有一个一致的选择，但是 Adam 优化器被广泛的使用。

LSTM单元的数量：这个值很大程度上取决于输入文本的平均长度。而更多的单元数量可以帮助模型存储更多的文本信息，当然模型的训练时间就会增加很多，并且计算成本会非常昂贵。

词向量维度：词向量的维度一般我们设置为50到300。维度越多意味着可以存储更多的单词信息，但是需要付出的是更昂贵的计算成本。

### 训练

基本思路：首先定义一个 TensorFlow 会话。然后加载一批评论和对应的标签。接下来，调用会话的 run 函数。这个函数有两个参数，第一个是 fetches 参数（定义了我们感兴趣的值）。第二个是 feed_dict 参数,就是提供的占位符。

目标：通过优化器来最小化损失函数。

做法：将一批处理的评论和标签输入模型，然后不断对这一组数据进行循环训练。

In [46]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())

for i in range(iterations):
    #Next Batch of reviews
    nextBatch, nextBatchLabels = getTrainBatch();
    sess.run(optimizer, {input_data: nextBatch, labels: nextBatchLabels}) 
    
    if (i % 1000 == 0 and i != 0):
        nextBatch, nextBatchLabels = getTestBatch();
        loss_ = sess.run(loss, {input_data: nextBatch, labels: nextBatchLabels})
        accuracy_ = sess.run(accuracy, {input_data: nextBatch, labels: nextBatchLabels})
        print("iteration {}/{}...".format(i+1, iterations),
              "loss {}...".format(loss_),
              "accuracy {}...".format(accuracy_))  
        
    #Save the network every 10,000 training iterations
    if (i % 10000 == 0 and i != 0):
        save_path = saver.save(sess, "models/pretrained_lstm.ckpt", global_step=i)
        print("saved to %s" % save_path)

iteration 1001/50000... loss 0.7066070437431335... accuracy 0.5833333134651184...
iteration 2001/50000... loss 0.6860122680664062... accuracy 0.375...
iteration 3001/50000... loss 0.6711758971214294... accuracy 0.4583333432674408...
iteration 4001/50000... loss 0.660219669342041... accuracy 0.625...
iteration 5001/50000... loss 0.7247483730316162... accuracy 0.5833333134651184...
iteration 6001/50000... loss 0.7750604152679443... accuracy 0.5...
iteration 7001/50000... loss 0.675737202167511... accuracy 0.4583333432674408...
iteration 8001/50000... loss 0.7090887427330017... accuracy 0.6666666865348816...
iteration 9001/50000... loss 0.5143955945968628... accuracy 0.75...
iteration 10001/50000... loss 0.556067168712616... accuracy 0.75...
saved to models/pretrained_lstm.ckpt-10000
iteration 11001/50000... loss 0.6969477534294128... accuracy 0.625...
iteration 12001/50000... loss 0.6766869425773621... accuracy 0.7083333134651184...
iteration 13001/50000... loss 0.4536910057067871... acc

导入一个预训练的模型需要使用 TensorFlow 的另一个会话函数，称为 Saver ，然后利用这个会话函数来调用 restore 函数。这个函数包括两个参数，一个表示当前的会话，另一个表示保存的模型。

In [47]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()   # 实例化
saver.restore(sess, tf.train.latest_checkpoint('models'))   # 根据时间记录找出最终模型

INFO:tensorflow:Restoring parameters from models\pretrained_lstm.ckpt-40000


然后从测试集中导入一些电影评论。请注意，这些评论是模型从来没有看见过的。

In [49]:
iterations = 10
for i in range(iterations):
    nextBatch, nextBatchLabels = getTestBatch();
    print("Accuracy for this batch:", (sess.run(accuracy, {input_data: nextBatch, labels: nextBatchLabels})) * 100)

Accuracy for this batch: 70.83333134651184
Accuracy for this batch: 66.66666865348816
Accuracy for this batch: 87.5
Accuracy for this batch: 87.5
Accuracy for this batch: 95.83333134651184
Accuracy for this batch: 75.0
Accuracy for this batch: 83.33333134651184
Accuracy for this batch: 70.83333134651184
Accuracy for this batch: 83.33333134651184
Accuracy for this batch: 66.66666865348816
